In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import gc
from datasets import load_dataset, DatasetDict, Dataset
import numpy as np
from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support, accuracy_score, classification_report, matthews_corrcoef
import pandas as pd
import wandb
import os

/home/mike/miniforge3/envs/modern/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
os.environ["WANDB_PROJECT"] = "DEBATE-ModernBERT-Large"

modname = "MoritzLaurer/ModernBERT-large-zeroshot-v2.0"
training_directory ='training_ModernLarge'
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

In [4]:
ds = load_dataset("mlburnham/Pol_NLI")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(modname)
id2label = {0: "entailment", 1: "not_entailment"}
label2id = {"entailment":0, "not_entailment":1}

def tokenize_function(docs):
    return tokenizer(docs['premise'], docs['augmented_hypothesis'], padding = False, truncation = False)
def model_init():
  return AutoModelForSequenceClassification.from_pretrained(modname, 
                                                           num_labels=2,
                                                           ignore_mismatched_sizes=True,
                                                           label2id = label2id, 
                                                           id2label = id2label)

dstok = ds.map(tokenize_function, batched = True)
dstok = dstok.rename_columns({'entailment':'label'})

training_args = TrainingArguments(output_dir=training_directory,
    logging_dir=f'{training_directory}/logs',
    lr_scheduler_type= "linear",
    group_by_length=True,
    learning_rate=9e-6 if "large" in modname else 2e-5,
    per_device_train_batch_size=4 if "large" in modname else 16,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4 if "large" in modname else 1,
    num_train_epochs=10,
    warmup_ratio=0.06,
    weight_decay=0.01,
    fp16=True,
    fp16_full_eval=True,
    eval_strategy="epoch",
    seed=1,
    save_strategy="epoch",
    dataloader_num_workers = 1
)

def compute_metrics_standard(eval_pred, label_text_alphabetical=list(id2label.values())):
    labels = eval_pred.label_ids
    pred_logits = eval_pred.predictions
    preds_max = np.argmax(pred_logits, axis=1)

    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(labels, preds_max, average='macro')
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(labels, preds_max, average='micro')
    acc_balanced = balanced_accuracy_score(labels, preds_max)
    acc_not_balanced = accuracy_score(labels, preds_max)
    mcc = matthews_corrcoef(labels, preds_max)

    metrics = {'MCC': mcc,
            'f1_macro': f1_macro,
            'f1_micro': f1_micro,
            'accuracy_balanced': acc_balanced,
            'accuracy': acc_not_balanced,
            'precision_macro': precision_macro,
            'recall_macro': recall_macro,
            'precision_micro': precision_micro,
            'recall_micro': recall_micro,
            }
    print("Aggregate metrics: ", {key: metrics[key] for key in metrics if key not in ["label_gold_raw", "label_predicted_raw"]} ) 
    print("Detailed metrics: ", classification_report(
        labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),
        target_names=label_text_alphabetical, sample_weight=None,
        digits=2, output_dict=True, zero_division='warn'),
    "\n")

    return metrics

trainer = Trainer(
    model_init=model_init,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=dstok['train'],
    eval_dataset=dstok['validation'],
    compute_metrics=lambda x: compute_metrics_standard(x, label_text_alphabetical=list(id2label.values()))
)

In [10]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mlburnham. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss,Mcc,F1 Macro,F1 Micro,Accuracy Balanced,Accuracy,Precision Macro,Recall Macro,Precision Micro,Recall Micro
0,0.732000,0.182307,0.893737,0.946868,0.948191,0.946724,0.948191,0.947013,0.946724,0.948191,0.948191
1,0.476600,0.204826,0.916039,0.957974,0.959098,0.956987,0.959098,0.959055,0.956987,0.959098,0.959098
2,0.255200,0.291375,0.905836,0.952747,0.954110,0.950857,0.954110,0.954989,0.950857,0.954110,0.954110
3,0.099100,0.307700,0.906207,0.952824,0.954243,0.950416,0.954243,0.955807,0.950416,0.954243,0.954243
4,0.072600,0.312559,0.911532,0.955730,0.956903,0.954854,0.956903,0.956680,0.954854,0.956903,0.956903
5,0.045400,0.374698,0.911276,0.955364,0.956704,0.952971,0.956704,0.958320,0.952971,0.956704,0.956704
6,0.009900,0.356787,0.919876,0.959918,0.960960,0.959260,0.960960,0.960617,0.959260,0.960960,0.960960
7,0.015100,0.478835,0.914809,0.957358,0.958500,0.956363,0.958500,0.958449,0.956363,0.958500,0.958500
8,0.000000,0.505373,0.921822,0.960786,0.961891,0.959145,0.961891,0.962684,0.959145,0.961891,0.961891
9,0.003600,0.565398,0.920444,0.960109,0.961226,0.958549,0.961226,0.961902,0.958549,0.961226,0.961226


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Aggregate metrics:  {'MCC': np.float64(0.8937372016784191), 'f1_macro': 0.9468676111277339, 'f1_micro': 0.9481910082468742, 'accuracy_balanced': np.float64(0.9467243166518844), 'accuracy': 0.9481910082468742, 'precision_macro': 0.9470129316278186, 'recall_macro': 0.9467243166518844, 'precision_micro': 0.9481910082468742, 'recall_micro': 0.9481910082468742}
Detailed metrics:  {'entailment': {'precision': 0.939595192915876, 'recall': 0.9373718252090235, 'f1-score': 0.9384821922135355, 'support': 6339.0}, 'not_entailment': {'precision': 0.9544306703397613, 'recall': 0.9560768080947453, 'f1-score': 0.9552530300419323, 'support': 8697.0}, 'accuracy': 0.9481910082468742, 'macro avg': {'precision': 0.9470129316278186, 'recall': 0.9467243166518844, 'f1-score': 0.9468676111277339, 'support': 15036.0}, 'weighted avg': {'precision': 0.9481762082893483, 'recall': 0.9481910082468742, 'f1-score': 0.9481826429047809, 'support': 15036.0}} 



...

Aggregate metrics:  {'MCC': np.float64(0.921821926774226), 'f1_macro': 0.9607855911921364, 'f1_micro': 0.9618914604948124, 'accuracy_balanced': np.float64(0.9591448950116943), 'accuracy': 0.9618914604948124, 'precision_macro': 0.9626838248196701, 'recall_macro': 0.9591448950116943, 'precision_micro': 0.9618914604948124, 'recall_micro': 0.9618914604948124}
Detailed metrics:  {'entailment': {'precision': 0.9671095268956578, 'recall': 0.9416311721091655, 'f1-score': 0.9542003037327152, 'support': 6339.0}, 'not_entailment': {'precision': 0.9582581227436823, 'recall': 0.9766586179142233, 'f1-score': 0.9673708786515575, 'support': 8697.0}, 'accuracy': 0.9618914604948124, 'macro avg': {'precision': 0.9626838248196701, 'recall': 0.9591448950116943, 'f1-score': 0.9607855911921364, 'support': 15036.0}, 'weighted avg': {'precision': 0.9619897701844492, 'recall': 0.9618914604948124, 'f1-score': 0.9618183198320216, 'support': 15036.0}} 



/tmp/ipykernel_18792/3974363994.py:27: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before t

Aggregate metrics:  {'MCC': np.float64(0.9204441364123934), 'f1_macro': 0.9601085447709636, 'f1_micro': 0.9612263899973397, 'accuracy_balanced': np.float64(0.9585485984173163), 'accuracy': 0.9612263899973397, 'precision_macro': 0.9619016453109515, 'recall_macro': 0.9585485984173163, 'precision_micro': 0.9612263899973397, 'recall_micro': 0.9612263899973397}
Detailed metrics:  {'entailment': {'precision': 0.9656957928802589, 'recall': 0.9414734185202713, 'f1-score': 0.9534307852064862, 'support': 6339.0}, 'not_entailment': {'precision': 0.958107497741644, 'recall': 0.9756237783143613, 'f1-score': 0.9667863043354412, 'support': 8697.0}, 'accuracy': 0.9612263899973397, 'macro avg': {'precision': 0.9619016453109515, 'recall': 0.9585485984173163, 'f1-score': 0.9601085447709636, 'support': 15036.0}, 'weighted avg': {'precision': 0.9613066333417823, 'recall': 0.9612263899973397, 'f1-score': 0.9611557752214185, 'support': 15036.0}} 



/tmp/ipykernel_18792/3974363994.py:27: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),


TrainOutput(global_step=107050, training_loss=0.19017183457118003, metrics={'train_runtime': 23024.6099, 'train_samples_per_second': 74.394, 'train_steps_per_second': 4.649, 'total_flos': 1.1064603257603529e+18, 'train_loss': 0.19017183457118003, 'epoch': 9.999929944188871})